# Webscraping von Stellenanzeigen – Korpuserstellung mit Hindernissen

**TU Dresden, Institut für Germanistik, Professur für Angewandte Linguistik**

***

**Seminar:** Programmieren lernen für Geistes- und Sozialwissenschaftler:innen

**Dozent:** Yannick Frommherz

**Semester der Prüfungsleistung:** Wintersemester 2023/2024

**Art der Prüfungsleistung:** Kombinierte Arbeit, Vertiefungsmodul Europäische Sprachen

***

**Eingereicht von:** Anna Bakker, Master EuroS, Matr.-Nr. 4068462

**Datum:** 31.03.2024

***

## Worum geht's?
***

Die vorliegende Projektarbeit soll den Aufbau eines geeigneten Skripts zeigen, mit dem Websites gescraped und somit beliebig große Textmengen gesammelt werden können, um zunächst ein nach eigenen Parametern aufgebautes Textkorpus zu erstellen. Als konkreter Anwendungsfall werden die Titel und Texte diverser Stellenanzeigen des Jobportals StepStone extrahiert und in einer Datei zusammengestellt. Die Textmenge soll sodann von irrelevanten Zeichen bereinigt und so aufbereitet werden, dass ein zur Untersuchung entsprechender Fragestellungen geeignetes Korpus entsteht. Dieses kann dann mithilfe der Mittel der Datenanalyse auf verschiedene Aspekte hin untersucht und modifiziert werden. In diesem Beispiel sollen die verschiedenen Arten, Stellenanzeigen zu Gendern, untersucht werden. Da dies erst einmal primär die Titel betrifft, müssen Titel und Textkörper getrennt voneinander analysierbar sein. Zudem muss beachtet werden, dass bei der Bereinigung von Sonderzeichen nicht die gendermarkierenden Zeichen mit entfernt werden. Im Anschluss werden dann noch einige Ideen für die Analyse vorgestellt und das Projekt dann mit einer kurzen Reflexion abgeschlossen. Das Skript ist in verschiedene Codeblöcke unterteilt, sodass die einzelnen Schritte nicht notwendigerweise alle nacheinander stattfinden müssen und die Codeblöcke zur Bereinigung und Aufbereitung von Texten auch unabhängig vom Scraping-Teil ausgeführt werden können. Dies hat noch den weiteren Vorteil, dass dadurch auch die Anzahl der Anfragen an die Website minimiert werden kann, was relevant ist, um Blockierungen durch die Website vorzubeugen. 


## Erste Hälfte – Der lange Weg zum Rohtext
***

Der erste Teil besteht darin, die Website über das Modul *requests* aufzurufen, den html-Quellcode mit *Beautiful Soup* zu parsen, die gewünschten Textbestandteile zu extrahieren und schließlich in eine Datei zu überführen, welche dann weiterverarbeitet werden kann. Da dieser Schritt erst einmal auf den gesamten Quellcode der Seite zugreift, geht es danach darum, sich mit dem Quellcode der Website und dessen html-Struktur genauer vertraut zu machen. Dabei muss dem Tagging besondere Aufmerksamkeit gewidmet werden, um herauszufinden, welche Teile des umfangreichen Quelltextes die spezifischen Titel und Textpassagen der Stellenanzeigen enthalten und welche Links jeweils darauf verweisen. Sobald die Linkstruktur identifiziert ist, kann ein geeigneter Regulärer Ausdruck (*RegEx*) gefunden werden, mit welchem sämtliche relevante Links aus dem Quelltext extrahiert und im Anschluss wieder "zusammengebaut" werden. Nun wird über diese Links iteriert, um auf die einzelnen Anzeigen zuzugreifen und – nach Identifikation der entsprechenden Tags – die jeweiligen Abschnitte der konkreten Stellenbeschreibungen zu entnehmen und in ein Dokument zu überführen. Da nicht willkürlich alle Anzeigen der gesamten Website abgerufen werden sollen, sondern nur diejenigen, welchen den gewünschten Suchkriterien entsprechen, werden Parameter eingeführt, welche so auch als Suchfilter auf der Website selbst zur Verfügung stehen: Stadt und Radius sowie die Angabe, bis zu welcher Seite die Anzeigen abgerufen werden sollen. Natürlich könnten auch weitere Parameter, wie die Berufsbezeichnung, implementiert werden. Da das Ziel der vorliegenden Arbeit jedoch ein Überblick über sämtliche Berufsgruppen schaffen soll, ist dies erst einmal irrelevant. Sämtliche Such- und Filtermöglichkeiten, welche die Website bietet, könnten auch im Code abgebildet werden und somit die Abfrage modifizieren.

Für dieses Vorgehen werden diverse Module benötigt, die zunächst importiert werden müssen: *re* für die RegEx, welche die Grundstruktur der gesuchten Links abbilden können und so ein Suchmuster für eine effiziente Ausgabe bieten. Zudem *BeautifulSoup*, eine Programmbibliothek, mit welcher sich html-Dokumente parsen lassen. *requests* ermöglicht erst das Senden von HTTP-Anfragen und eine Aufbereitung der entsprechenden Antworten der Website. Das letzte Modul, *time*, gehört in den Bereich des Troubleshootings und wird im weiteren Verlauf erläutert.

Beim Webscraping kann es zu verschiedenen Problemen kommen, die es im Blick zu behalten gilt. Viele Seiten haben Maßnahmen implementiert, um automatisiertes Zugreifen zu blockieren. Dies umfasst unter anderem rate limiting, Bot-Erkennung oder das Blocken von IP-Adressen (vgl. Emre: 2023). Ein nicht unwesentlicher Teil dieses ersten Blocks wird sich also auch mit Vorbeugung dieser Effekte sowie mit Troubleshooting und Lösungsansätzen befassen.

### Konstruktion der Links und Modifikation der Abfrage

Zuerst müssen die bereits beschriebenen Module importiert werden. Als nächstes wird eine Funktion definiert, welche die für diese Arbeit relevanten Parameter Stadt, Radius und Seitenzahl enthält. In diesem Schritt wäre es möglich, die unterschiedlichsten Parameter einzufügen, wenn die dafür zuständige Kennzeichnung im Link der Website erkennbar ist. Diese Stelle wird dann durch einen Platzhalter in der Funktion ersetzt, mit welchem auch festgelegt wird, welcher Zeichentyp eingetragen werden muss: in diesem Falle ein String für die Stadt und Ganzzahlen für Radius und Seitenzahl. 
Über *requests* wird dann die URL aufgerufen und auf den Quelltext zugegriffen. Dieser Schritt birgt das meiste Problempotential, da an dieser Stelle die Blockmechanismen und Fallen der Website greifen. Um das zu umgehen, wurde ein Fake-Header in die Abfrage eingebaut (Ikleiw: 2020), welcher im folgenden Abschnitt definiert wird und dafür sorgt, dass Herkunft und Art der Anfrage verschleiert und eine menschliche Nutzung nachgeahmt wird. Über einen passenden RegEx werden dann alle Links zu den gewünschten Anzeigen gefunden, ausgegeben und der Liste "matches" hinzugefügt.

In [5]:
import re
import os
from bs4 import BeautifulSoup
import requests 
import time

def search_urls(city, radius, page):
    url = "https://www.stepstone.de/jobs/in-%s?radius=%i&page=%i" % (city, radius, page) #abzurufende URL mit entsprechenden Platzhaltern
    print(url)
    quelltext = requests.get(url, headers=headers).text #http-Anfrage unter falschem Header

    #RegEx, welcher die gewünschten Links matcht; diese werden dann gesucht und ausgegeben
    regex = r'href="([^"]*\/stellenangebote-[^\s"]+)"' 
    url_matches = re.findall(regex, quelltext)
    print(url_matches)
    return url_matches

#die Variablen werden initiiert und die gewünschten Parameter eingetragen
cities = ["Würzburg"] 
r = 5
til_page = 1 
#menschliche Nutzung nachahmender Fake-Header
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "de-DE,de;q=0.5","Accept-Encoding": "gzip, deflate"}
matches = [] 
for c in cities:
   print(c)
for p in range(1,til_page+1):
        print(p)
        found_urls = search_urls(c, r, p)
        matches.extend(found_urls) #die gefundenen URLs werden der matches-Liste hinzugefügt


for match in matches:
     print(match)

#entfernt eventuelle Dopplungen in den Links
matches = list(set(matches))
print(len(matches))
#Stamm-URL, Stadt, Seitenzahl und entsprechende Matches werden ausgegeben


Würzburg
1
https://www.stepstone.de/jobs/in-Würzburg?radius=5&page=1
['/stellenangebote--Account-Managerin-Deutschland-Region-Sued-m-w-d-Wuerzburg-Truck-line-GmbH--10928444-inline.html', '/stellenangebote--Aussendienstmitarbeiter-als-selbststaendiger-Handelsvertreter-m-w-d-Nuernberg-Wuerzburg-Stroeer-Media-Deutschland-GmbH--10887084-inline.html', '/stellenangebote--Vertriebsmitarbeiter-Aussendienst-in-der-Neukundenakquise-m-w-d-Wuerzburg-Stroeer-Media-Deutschland-GmbH--10887087-inline.html', '/stellenangebote--Elektroniker-Schutz-und-Stationsleittechnik-Strom-m-w-d-Wuerzburg-Wuerzburger-Versorgungs-und-Verkehrs-GmbH--10887353-inline.html', '/stellenangebote--Vertriebsmitarbeiter-Aussendienst-in-der-Neukundenakquise-m-w-d-Wuerzburg-Stroeer-Media-Deutschland-GmbH--10887086-inline.html', '/stellenangebote--Chefarzt-m-w-d-Gynaekologie-Geburtshilfe-fuer-die-Klinik-fuer-Gynaekologie-Geburtshilfe-Wuerzburg-Klinikum-Wuerzburg-Mitte-gGmbH--10928083-inline.html', '/stellenangebote--Vermoegenskun

### Der Problemlösung erster Teil: Finde den Fehler

Es werden also der Linkstamm sowie die jeweiligen Erweiterungen der Links für alle den Parametern entsprechenden Stellenanzeigen ausgegeben sowie die Anzahl der gefundenen matches, um überprüfen zu können, ob alles gefunden wurde.  
Was in der Theorie gut klingt, führte in der Umsetzung zu diversen Problemen, welche nicht immer identifiziert werden konnten und manchmal von einer Ausführung des Codes zur nächsten scheinbar willkürlich wechselten. So kam es beim ersten Versuch zu gar keinem Ergebnis, die Anfrage lief unbegrenzt weiter, bis zum Abbruch. Es wurde keine Fehlermeldung ausgegeben. Auch eine versuchte Ausgabe des Status Codes oder Inhalte der response zur Identifikation des Problems verliefen erfolglos. Ein Testdurchlauf mit einer anderen Website ergab, dass es nicht am Code lag, da dort die Ausführung das gewünschte Ergebnis lieferte. Ein weiterer Versuch einige Tage später ergab eine lange Fehlermeldung mit dem Hinweis, die Verbindung sei unerwartet unterbrochen worden. Beim dritten Versuch schließlich enthielt die Ausgabe nur die Stammlinks der Übersichtsseiten, nicht jedoch die der Stellenanzeigen. Es kann angenommen werden, dass diese Ergebnisse durch Blockingmechanismen der Website zustande kamen. Deswegen wurde zunächst das *time*-Modul implementiert, welches Pausen von beliebig vielen Sekunden zwischen die Abfragen einbaut, um zu kaschieren, dass es sich um ein automatisiertes Abrufen handelt.


In [ ]:
#Codeschnipsel, die zum Troubleshooting verwendet wurden:

response = requests.get(url) #Überprüfung, ob überhaupt auf die URL zugegriffen werden kann

#Falls nicht, können der Status Code, die Response Header und der Response Content ausgegeben werden, um das Problem zu identifizieren:
print('Response Status Code:', response.status_code)

print('Response Headers:')
for header, value in response.headers.items():
    print(f'{header}: {value}')

print('Response Content:', response.text)

time.sleep(10) #baut eine Pause von (x) Sekunden zwischen den Anfragen ein


Nachdem das Problem weiterhin nicht identifiziert werden konnte, sollte es vom anderen Ende her betrachtet werden: der IP, von der aus die Anfrage gesendet wird und welche entsprechend von der Website geblockt wird. Nachdem der Zugriff über einen klassischen VPN-Client ebenfalls keine Erfolge erzielte, funktionierte die Abfrage über das Uninetzwerk *eduroam* zwischenzeitlich interessanterweise auch ohne weitere Schritte zum Umgehen der Blockierung. Somit bot sich die Nutzung des OpenVPN-Zugangs zum Uninetzwerk an, welches es ermöglichte, auch über andere Internetverbindungen erfolgreiche Abfragen generieren zu können. Warum das temporär funktionierte, konnte unbefriedigenderweise leider bis zum Zeitpunkt der Abgabe dieser Arbeit nicht ergründet werden.

### Der Problemlösung zweiter Teil: Der Kompromiss

Nach einigen Tagen wurden dann jedoch auch die Anfragen über das Uni-VPN blockiert und die Verbindung während der Abfragen abgebrochen, sodass keine Ausgabe möglich war. Eine Suche bei *stack overflow* ergab, dass ein Fake-Header in die Anfrage eingebaut werden kann, um zu verschleiern, dass es sich um ein Skript handelt. Dieser wird initialisiert und dann für die HTTP-Anfrage mit abgerufen. In Kombination mit der Nutzung des Uni-VPN führte diese Strategie zu einem Teilerfolg. Die Anfragen wurden nun zwar nicht mehr geblockt und es wurden die gewünschten Links und der Quellcode ausgegeben, der neue Header führte jedoch zu neuen Problemen: die Ausgabe entsprach nicht mehr den Anzeigen, welche auf der Website selbst bei gleicher Anfrage angezeigt wurden, die Reihenfolge wurde vertauscht, Teile fehlten. Durch Probieren konnte herausgefunden werden, dass die jeweils erste Seite der Anzeigen (also til_page=1 im ersten Block) korrekt ausgegeben wird. Beim Abruf mehrerer Seiten kommt es dann jedoch zu den genannten Fehlern. Häufig wurde auch einfach mehrfach der Quellcode der ersten Seite ausgegeben oder es führte zum Abbruch der Anfrage. Das Problem scheint also im korrekten Durchblättern der Seiten zu liegen. Leider konnte dafür keine Lösung gefunden werden. 

So muss der Weg also um das Problem herum führen, falls für das gewünschte Korpus nicht bloß 25 Anzeigen pro Stadt ausreichen, denn das ist die Anzahl pro Seite. Eine größere Menge Anzeigen könnte also entweder abgerufen werden, indem die Zahl der Städte erhöht wird und dann jeweils die erste Seite pro Stadt abgerufen wird. Diese Möglichkeit kann mit dem ersten Block des Skripts automatisiert und fehlerfrei in kurzer Zeit ausgeführt werden. Sollen jedoch mehr als 25 Anzeigen einer Stadt abgerufen werden, muss zu einer etwas uneleganten und auch zeitintensiveren Methode gegriffen werden. Das automatisierte Identifizieren und Sammeln der Anzeigenlinks im ersten Block fiele in diesem Falle komplett weg und kann gelöscht oder auskommentiert werden, da zumindest die hier untersuchte Seite StepStone offenbar keine größeren Abfragen erlaubt. Es muss beachtet werden, dass die für den weiteren Verlauf benötigten Module dann ebenfalls nicht importiert werden und dies dann in den folgenden Codeblöcken geschehen muss. 

Um wenigstens die Extraktion und Bereinigung des Quelltexts sowie die Analyse automatisiert durchführen zu können, können die Links, über welche iteriert werden soll, manuell gesammelt und eingelesen werden, um das Problem des ersten Blocks zu umgehen. Dazu müssen sie nach der entsprechenden Suchanfrage direkt von der Website kopiert und in ein Textdokument eingefügt werden. Wie der Code dafür angepasst werden muss, wird im übernächsten Block gezeigt. Der Rest des Skripts kann dann wie beabsichtigt genutzt werden. Zunächst jedoch die Version für den ursprünglich intendierten Vorgang zur Iteration der automatisch gesammelten Links und der Extraktion der relevanten Quelltextbestandteile:

In [6]:
#Definition der Funktion für die gewünschten Textblöcke
def extract(soup_offer_text):
    title = ""
    introduction = ""
    description = ""
    profile = ""
    benefits = ""
#nur dann Extraktion der Inhalte, wenn Titel vorhanden
    if (soup_offer_text.find(attrs={"data-at": "header-job-title"}) is None):
        return 
    else:
        title = str(soup_offer_text.find(attrs={"data-at": "header-job-title"}).contents[0]) 
    

#Greift auf den content innerhalb dieses Attributs zu; für jeden Anzeigenabschnitt einzeln
#Nimmt den Inhalt, wenn welcher da ist, aus den Attributen und dem span innerhalb der Attribute
    if (soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}).span is not None):
            introduction = str(soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}).span.contents[0]) 


    if (soup_offer_text.find(attrs={"data-at": "section-text-description-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-description-content"}).span is not None):
            description = str(soup_offer_text.find(attrs={"data-at": "section-text-description-content"}).span.contents[0])


    if (soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}).span is not None):
            profile = str(soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}).span.contents[0])


    if (soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}).span is not None):
            benefits = str(soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}).span.contents[0])

#Die Titel und Textabschnitte werden mit vorangstellter Doppelraute in die neu geöffnete Datei 'Rohtext' geschrieben, die Titel mit Dreifachslashes umgeben zur Separierung
    texts = [introduction, description, profile, benefits]
    text = open("rohtext.txt", "a", encoding='utf-16-le')
    text.write("\n" + "## ")
    text.write("///" + title+ "///" + "\n")
    for x in texts:   
        text.write(x + "\n")
    text.close()

text = open("rohtext.txt", "w", encoding='utf-16-le')
text.close()
i = 0
for url in matches:
    offer_url = "https://www.stepstone.de" + url #die extrahierten Linkbestandteile der Anzeigen werden durch den Stammlink komplettiert

    #Definition des Fake-Headers zur Verschleierung der Anfragen
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "de-DE,de;q=0.5","Accept-Encoding": "gzip, deflate"}

    sourcecode_offer = requests.get(offer_url, headers=headers).text
    soup_offer = BeautifulSoup(sourcecode_offer, "lxml")

    extract(soup_offer)
    i += 1
    
    #als Zusatz für die Übersichtlichkeit (und weil es cool ist :) ) wird der Fortschritt der Quelltextextraktion aus den Anzeigen dargestellt
    print(str(i) + "/" + str(len(matches)) + "---" + format((i/len(matches))* 100, '.2f') + "%") 


1/25---4.00%
2/25---8.00%
3/25---12.00%
4/25---16.00%
5/25---20.00%
6/25---24.00%
7/25---28.00%
8/25---32.00%
9/25---36.00%
10/25---40.00%
11/25---44.00%
12/25---48.00%
13/25---52.00%
14/25---56.00%
15/25---60.00%
16/25---64.00%
17/25---68.00%
18/25---72.00%
19/25---76.00%
20/25---80.00%
21/25---84.00%
22/25---88.00%
23/25---92.00%
24/25---96.00%
25/25---100.00%


### Iterieren über Links zur Extraktion der Quellcodeabschnitte

Da der Prozess der Extraktion der gewünschten Quelltextbestandteile mehrere, sich für jede Anzeige wiederholende Schritte umfasst, bietet es sich an, wieder eine Funktion dafür zu definieren. Diese greift nun den geparsten Quellcode auf und ermöglicht es, über die html-tags auf spezifische Bestandteile des Textes zuzugreifen und diese zu extrahieren. Wie eingangs erklärt, ist hierfür eine genaue Begutachtung der html-Struktur erforderlich. Dabei ergibt sich, dass alle Anzeigen dieselbe Struktur aufweisen und dieselben tags auf die entsprechenden Inhalte verweisen. Bei StepStone handelt es sich um die zu Beginn der Definition angeführten Kategorien, die zunächst initialisiert werden müssen. Dann wird zuerst geprüft, ob ein Titel vorhanden ist. Falls nicht, wird übersprungen, ansonsten wird der Titel extrahiert. Sämtliche für uns interessante Inhalte werden von dem Attribut "data-at" umspannt. Der Inhalt wird jeweils entnommen und in einen string gecastet. Dies geschieht für alle Kategorien, also neben dem Titel die Einleitung, Beschreibung, das Bewerber:innenprofil und die Vorteile in jeder Anzeige. Dann wird eine Textdatei geöffnet, in welcher die entnommenen strings jeweils mit einem Zeilenumbruch dazwischen hineingeschrieben werden. Die Titel werden jeweils von drei Slashes umrahmt, um sie vom Textkörper abzugrenzen. Jede neue Anzeige beginnt mit zwei Rauten. Bei jedem neuen Ausführen des Skripts wird die alte Textdatei überschrieben, sodass es nicht zu Dopplungen kommt. Der Fortschritt dieses Prozesses wird zum Schluss noch in ganzzahligen Anteilen sowie in Prozent angegeben. Das Ergebnis ist dann also eine Textdatei mit den extrahierten Inhalten aller Anzeigen, mit noch enthaltenen html-Tags, welches in der definierten Datei im Ordner, in welchem sich auch das Skript befindet, abgelegt wurde. Möglich wäre auch, den gesamten String einfach in ein Objekt zu überführen und damit weiterzuarbeiten. Eine Textdatei hat jedoch den Vorteil, dass man sie auch mit anderen Programmen weiterverarbeiten kann.


### Alternative: Manuelles Einlesen der Links

Der folgende Code zeigt nun die Variation des händischen Einlesens eines Dokuments mit zusammenkopierten Links, sollten die automatisierten Anfragen von der Website geblockt werden. Die zugrundeliegende Funktion ist die gleiche, es müssen in der zweiten Hälfte des Blocks jedoch einige Anpassungen vorgenommen werden:

In [ ]:
#Erneutes Importieren der benötigten Module, weil erster Codeblock nicht ausgeführt wird
import requests
from bs4 import BeautifulSoup 

def extract(soup_offer_text):
    title = ""
    introduction = ""
    description = ""
    profile = ""
    benefits = ""

    if (soup_offer_text.find(attrs={"data-at": "header-job-title"}) is None):
        return 
    else:
        title = str(soup_offer_text.find(attrs={"data-at": "header-job-title"}).contents[0]) 
    

    if (soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}).span is not None):
            introduction = str(soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}).span.contents[0]) 


    if (soup_offer_text.find(attrs={"data-at": "section-text-description-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-description-content"}).span is not None):
            description = str(soup_offer_text.find(attrs={"data-at": "section-text-description-content"}).span.contents[0])


    if (soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}).span is not None):
            profile = str(soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}).span.contents[0])


    if (soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}).span is not None):
            benefits = str(soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}).span.contents[0])

    texts = [introduction, description, profile, benefits]
    text = open("rohtext.txt", "a", encoding='utf-16-le')
    text.write("\n" + "## ")
    text.write("///" + title+ "///" + "\n")
    for x in texts:   
        text.write(x + "\n")
    text.close()

text = open("rohtext.txt", "w", encoding='utf-16-le')
text.close()
i = 0

#Das Textdokument mit den händisch kopierten Links wird eingelesen
with open("Links_manuell.txt") as read_file:
     file = read_file.read()
#der Gesamttext wird an den Zeilenumbrüchen getrennt, um auf die einzelnen Links zugreifen zu können    
for url in file.split('\n'): 
    offer_url = url 

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "de-DE,de;q=0.5","Accept-Encoding": "gzip, deflate"}
    
    #die Anzahl muss händisch definiert werden, da sie für die Berechnung nicht mehr aus dem ersten Block abgerufen werden kann
    matches = 500 

    sourcecode_offer = requests.get(url, headers=headers).text
    soup_offer = BeautifulSoup(sourcecode_offer, "lxml")

    extract(soup_offer)
    i += 1
    #die len-Funktion wird entfernt, da die Zahl zur Berechnung direkt übernommen wird und die Matches nicht mehr als Liste vorliegen
    print(str(i) + "/" + str(matches) + "---" + format((i/matches)* 100, '.2f') + "%") 

## Zweite Hälfte – vom Rohtext zur Analyse

Um am Ende ein gut analysierbares Textkorpus zu erhalten, muss die Textmenge noch bereinigt und von den html-Tags sowie überschüssigen Zeichen und Zeilen befreit werden. Dies geschieht zum großen Teil über RegEx, sodass das erforderliche Modul sicherheitshalber am Anfang noch einmal importiert wird, falls der erste Teil des Skripts nicht vorher ausgeführt wurde. Dann wird die Datei mit dem Rohtext eingelesen. Die folgenden Schritte zur Bereinigung sind auf den StepStone-Quelltext zugeschnitten, diesen Teil des Skripts könnte man jedoch mit kleinen Anpassungen auch auf beliebige andere Textmengen in html anwenden, die bereinigt werden sollen. 

In [7]:
import re 

#einlesen der im vorherigen Schritt erstellten Datei
text = open("rohtext.txt", "r", encoding='utf-16-le') 
lines = text.readlines()

korpus = "".join(lines)
    
#sämtliche html-Tags entfernen und durch Leerzeichen ersetzen
remove = ["<p>", "</p>", "<strong>", "</strong>", "<ul>", "</ul>", "<li>", "</li>", "<em>", "</em>", "<br/>", "</a>","<b>", "</b>", "<u>", "</u>", "<i>", "</i>"]
for x in remove:
  korpus = korpus.replace(x, " ")

#Hyperlink-Tags entfernen
korpus = re.sub(r'<a href=".*">', "", korpus)

#Zwischenüberschriften-Tags entfernen
korpus = re.sub(r'<h1>.*</h1>', "", korpus)

#Umbrüche durch Leerzeichen ersetzen
korpus = korpus.replace("\n", " ")

#entfernt zu viele und merkwürdig codierte Leerzeichen
korpus = korpus.replace("   ", " ")
while "  " in korpus:
    korpus = korpus.replace("  ", " ")

#ersetzt alles, was bei manueller Durchsicht an komischen Zeichen aufgefallen ist, durch die gängigen Equivalente
korpus = korpus.replace("„", "\"")
korpus = korpus.replace("“", "\"")
korpus = korpus.replace("·", "-")
korpus = korpus.replace("‑","-")
korpus = korpus.replace("’","'")
korpus = korpus.replace("…","...")
korpus = korpus.replace("\t"," ")

#Um auf Nummer sicher zu gehen, entfernt das hier nochmal alles, was nicht in den eckigen Klammern ist
korpus = re.sub(r'[^a-zA-Z0-9äöü.,;ß/()\[\]\-#\|&\*\+:ÖÄÜ?"!§$%&\\²³€\'\s]', "", korpus)

#der bereinigte String wird in eine neu geöffnete Datei geschrieben
text = open("Dachau.txt", "w", encoding='utf-16-le')   
text.write(korpus)
text.close()

## Ansätze für die quantitative Korpusanalyse

Nachdem die Anzeigentexte nach den gewünschten Parametern gescraped, zu einem Textkorpus zusammengefügt und bereinigt wurden, kann nun mit der eigentlichen Analyse begonnen werden. Dabei ist es wichtig, beim Preprocessing genau zu überlegen, was untersucht werden soll. Wenn es zum Beispiel um Lemmata geht, ist es sinnvoll, einheitliche Kleinschreibung und Bereinigung von sämtlichen Zeichen durchzuführen, während bei der Untersuchung von Genderformen eben dies kontraproduktiv wäre, da in dem Falle Genderzeichen und Binnenmajuskel zum Opfer fielen. 

Da es bei der Zusammenstellung dieses Korpus darum gehen sollte, verschiedene Arten des Genderns in Stellenanzeigen zu untersuchen, werden in den folgenden Codeblöcken nun einige Möglichkeiten der Weiterverarbeitung und Analyse gezeigt, um sich dem Untersuchungsgegenstand zu nähern. Ein zentraler Bestandteil stellt die Sortierung nach verschiedenen Genderarten dar. Für diesen Zweck eignet sich am Besten die Aufteilung in unterschiedliche dictionaries, eines für jede Art des Genderns. Zunächst werden jedoch die gesamten Stellenanzeigen in eine dictionary-Struktur gebracht, mit den Titeln als keys und den Jobbeschreibungen als values.

Im nächsten Schritt erfolgt nun die Aufteilung nach Arten des Genderns. Entsprechende RegEx filtern die Titel, welche die gewünschten Muster enthalten, heraus. Danach werden sie in dictionaries abgelegt und aus dem Gesamtdictionary (job_text) entfernt. Wichtig ist hierbei die Reihenfolge im Code: da viele Anzeigentitel mehrere Genderarten enthalten (zum Beispiel "Mitarbeiter* m/w/d") muss entschieden werden, in welches dictionary dieser Titel sortiert wird, oder eher gesagt: Welche Art des Genderns "relevanter" ist. Da ein einzelnes Sternchen deutlich seltener vorkommt als der Zusatz m/w/d, sollte das Beispiel also im Sternchen-dictionary landen. Aus diesem Grund muss die Sternchen-Sortierung vor der m/w/d-Sortierung stattfinden, da der Eintrag nur einem dictionary zugeordnet wird und zwar dem, dessen RegEx es als erstes matcht. Dieses Vorgehen ist natürlich ganz von den jeweiligen Fragestellungen und dem Textmaterial abhängig und kann entsprechend variiert werden.

Nachdem sämtliche Titel in die jeweils passenden dictionaries sortiert wurden, wird das Ergebnis dann entsprechend ausgegeben. Die Titel, sortiert nach ihrer Genderart, sowie ihre Anzahl und die Gesamtzahl der Jobs. Zudem wird auch der "Rest" ausgegeben, also alle Einträge, die nach der Sortierung noch im ursprünglichen dictionary verblieben sind. So können auch abweichende Varianten oder verrückte Sonderformen, die nicht von den RegEx gematcht werden, angezeigt werden.

In [8]:

import re

#Die entsprechende (Teil-)Korpusdatei wird eingelesen
korpus = open("Würzburg.txt", "r", encoding='utf-16-le') 
lines = korpus.readlines()

korpus = "".join(lines)
korpus = korpus.strip()

korpus = korpus.split("##")
print(len(korpus))
job_text = {} #das dictionary für alle Stellenanzeigentexte wird initiiert
korpus.remove('') #Leere Einträge werden entfernt

title_regex = r'///.*///'
for x in korpus:
    inseration = x.strip()
    title = str(re.findall(title_regex, inseration).pop())

    #Titel und Stellenbeschreibung werden getrennt, mit Titeln als keys und den Beschreibungen als values
    inseration = inseration.replace(title,"")
    inseration.strip()
    title = title.replace("///","")
    job_text[title] = inseration
num_jobs = len(job_text)


#Verschiedene dictionaries für die verschiedenen Genderformen
print("sternchen") #reines Sternchen ohne folgendes Suffix
sternchen_dict = {}
sternchen_regex =r'.\*(\s|\))'

for title in job_text:
    if len(re.findall(sternchen_regex, title)) != 0:
        print(title)
        sternchen_dict[title] = job_text[title]

#die gematchten Titel werden aus dem Gesamtdictionary entfernt und ein Trennstrich ausgegeben
for title in sternchen_dict:
    job_text.pop(title, None)
print("-----") 

print("in-Formen")
in_dict = {} # für alle *in, :in, _in
in_regex = r'\w*(\*|:|_)\s*in'

for title in job_text:
    if len(re.findall(in_regex, title)) != 0:
        print(title)
        in_dict[title] = job_text[title]

for title in in_dict:
    job_text.pop(title, None)
print("-----")     

print("m/d/w-Formen")
mwd_dict = {} #für alle (m/w/d), (w/m/d), etc. - Formen
mwd_regex = r'(\(|\s)(\w(\/|\||,)\s*\w(\/|\||,)\s*\w)|(\w(\/|\||,)\s*\w)(\)|\s)'

for title in job_text:
    
    if len(re.findall(mwd_regex, title)) != 0:
        print(title)
        mwd_dict[title] = job_text[title]


for title in mwd_dict:
    job_text.pop(title, None)
print("-----")

print("Rest")
for title in job_text:
    print(title)
print("Jobs insg.: " + str(num_jobs))
print("Jobs Sternchen: " + str(len(sternchen_dict)))
print("Jobs *_:in: " + str(len(in_dict)))
print("Jobs m/w/d: " + str(len(mwd_dict)))

145
sternchen
Software Developer* Cloud-Technologien
Cloud Architect* (AWS, Azure, GCP, private Cloud)
-----
in-Formen
Vertriebler:in / Berater:in von Immobilieneigentümer:innen / Sales Multimedia (Erfahrung im Außendienst)
Category Insights Manager:in DE (m/w/d)
Ingenieur*in als Sachverständige*r Anlagensicherheit zur Ausbildung (ZÜS Druck- und Ex-Anlagen, Notifizierte Stelle, Funktionale Sicherheit)
Projektingenieur*in für die Beleuchtung von Jugendspieleinrichtungen (w/m/d)
Versuchsingenieur*in für Sicherheitsprüfungen
Ingenieur*in / Techniker*in im Bereich Versorgungstechnik
Ingenieur*in als Gutachter*in im Bereich Anlagensicherung
Sachbearbeiter*in HR Vertragsmanagement
Junior Versuchstechniker*in für Sicherheitsprüfungen
Projektingenieur*in im Klärwerksbau (w/m/d)
Gleisbauer:in / Fachlagerist:in
-----
m/d/w-Formen
Elektrokonstrukteur (m/w/d)
SAP Analytics Cloud (SAC) Junior Consultant (m/w/d)
Mitarbeiter/in (m/w/d) für die Kundenbetreuung im Außendienst Region Tübingen
Mitarbeite

Dies ist natürlich nur ein einziges Beispiel für einen Analyseansatz. Je nach Fragestellungen können sämtliche Skriptblöcke beliebig angepasst werden und bieten umfassende Untersuchungsmöglichkeiten, wie Lemmataanalysen, Wortformen und deren relative Anteilen, Kollokationen und Ähnliches. Durch die Strukturierung des Korpus als dictionary sind auch Kookurrenzen und Auffälligkeiten zwischen den Titel und Textbestandteilen möglich, beispielsweise, ob es Zusammenhänge gibt zwischen der Genderform des Titels und der verwendeten Lexik der Texte. "Sind die Texte unter dem mit dem Mindeststandard m/w/d gegenderten Titel im 'generischen' Maskulinum verfasst?" "Bedeuten Sternchenformen im Titel auch Sternchenformen im Text?" wären mögliche Fragestellungen, denen so nachgegangen werden könnte.   

## Fazit
Die vorliegende Arbeit sollte einen beispielhaften Weg nachzeichnen, wie ein einfaches Programm geschrieben werden kann, um Texte eines Online-Stellenportals automatisiert zu extrahieren und schließlich ein Textkorpus zu erstellen, welches bereinigt und analysiert wird. Dabei ist zu beachten, dass die erste Hälfte des Programms sehr spezifisch auf die Website StepStone zugeschnitten ist und erst einmal nicht generalisiert für alle Stellenportale angewendet werden kann. Durch entsprechende Analyse des jeweiligen Quellcode-Aufbaus der Seiten sowie der html-Strukturen kann der Code jedoch entsprechenden angepasst werden und somit als Vorlage dienen. Die zweite Hälfte kann flexibel für simple Textdokumente unabhängig von ihrer Quelle verwendet werden und macht den Code somit auch ohne den Bestandteil des Scrapings vielseitig einsetzbar.

Wie gezeigt wurde, sind die Abwehrmechanismen solcher Websites nicht zu unterschätzen. StepStone ist ein Beispiel dafür, wie viele, teils schwer zu durchschauende Fallen im Code der Seite eingebaut sein können, welche einen automatisierten Zugriff behindern oder unmöglich machen. Dies ist bei der Planung des Projekts zu berücksichtigen. Zudem handelt es sich beim Webscraping um eine rechtliche Grauzone, bei der es zu Schwierigkeiten kommen kann, wenn die gesammelten Daten zum Beispiel veröffentlicht oder für kommerzielle Zwecke genutzt werden. Auch hier empfiehlt es sich, vorher Nachforschungen bezüglich der Bedingungen, Möglichkeiten und Grenzen der jeweiligen Website anzustellen.

Sind diese Hürden überwunden, bietet ein solches Programm aber einen enorm zeitsparenden und effizienten Weg, große Textmengen nach sehr individuellen Kriterien zu sammeln sowie schnell und zu sauber bereinigen und – last but not least – auch nach den verschiedensten Gesichtspunkten quantitativ analysieren zu können. 

## Bibliographie

### Primärquellen

https://www.stepstone.de/, Stand 01.04.2024.

### Sekundärquellen 

Emre (2023): Wie man Webscraping verhindert: Schützen Sie Ihre Website. https://de.webscraping.blog/wie-man-web-scraping-verhindert/, Stand 18.03.2024.

Fett, Daniel (2006): Tutorial Reguläre Ausdrücke. https://danielfett.de/2006/03/20/regulaere-ausdruecke-tutorial/, Stand 20.03.2024.

Ikleiw, Aleksander (2020): *Antwort auf einen Beitrag* (ohne Titel) bei www.stackoverflow.com. https://stackoverflow.com/questions/62973647/get-request-works-with-postman-but-not-with-python-requests-and-curl, Stand 20.03.2024.

McKinney, Wes (2023): Datenanalyse mit Python. Auswertung von Daten mit pandas, NumPy und Jupyter. 3. Auflage. Heidelberg: O'Reilly.

pandas (2024): Working with text data. https://pandas.pydata.org/docs/user_guide/text.html, Stand 19.03.2024.

Python Software Foundation (2001–2024): Python HOWTOs. https://docs.python.org/3/howto/index.html, Stand 01.04.2024.

Richardson, Leonard (2004–2023): Beautiful Soup Documentation. https://www.crummy.com/software/BeautifulSoup/bs4/doc/, Stand 10.03.2024.



### Tools 

``ChatGPT``

Prompts: 

+ How to prevent blocking mechanisms for web scraping?

+ Why does the http-request takes so long without giving an error?

+ Why does the following RegEx ^href=".*stellenangebote-\S+"$ match the searched link in a trial when the link is seperately tested but not when I try to apply it to the whole source code?

+ "extract(soup_offer)
    i += 1
    print(str(i) + "/" + str(len(matches)) + "---" + format((i/len(matches))* 100, '.2f') + "%")?"
    Why does it add up to 125/25 and therefore calculates a progress of 500%?

``RegExr``

+ diverses Durchprobieren sämtlicher verwendeter RegEx